In [1]:
import sys, os
from importlib import reload

import time
import popDMS


# GLOBAL VARIABLES

## Pointers to data and output
OUT_DIR  = 'output/'                                         # general directory for output
FREQ_DIR = os.path.join(OUT_DIR, 'variant_frequencies/')     # directory for experimental variant (codon/aa) frequencies
SEL_DIR  = os.path.join(OUT_DIR, 'selection_coefficients/')  # directory for inferred selection coefficients
DATA_DIR = 'data/'                                           # general directory for external data
RAW_DIR  = os.path.join(DATA_DIR, 'raw_data/')               # unprocessed data directory
FIG_DIR  = 'figures/'                                        # directory for figures

## popDMS parameters
CORR_CUTOFF_PCT = 0.1  # maximum fraction of drop in correlation per log10(gamma) for setting optimal regularization strength

In [ ]:
import pandas as pd

root = 'Spike'
n_reps = 2

for f in [os.path.join(FREQ_DIR, '_'.join([root, 'single_aa_rep_%d.csv' % i])) for i in range(1, 1+n_reps)]:
    df = pd.read_csv(f)
    df.to_csv(f + '.gz', index=False, compression='gzip')

for f in [os.path.join(FREQ_DIR, '_'.join([root, 'double_aa_rep_%d.csv' % i])) for i in range(1, 1+n_reps)]:
    df = pd.read_csv(f)
    df.to_csv(f + '.gz', index=False, compression='gzip')

for f in [os.path.join(SEL_DIR, '_'.join([root, 'selection_coefficients.csv']))]:
    df = pd.read_csv(f)
    df.to_csv(f + '.gz', index=False, compression='gzip')

for f in [os.path.join(OUT_DIR + 'merged_preference/', root+'.csv')]:
    df = pd.read_csv(f)
    df.to_csv(f + '.gz', index=False, compression='gzip')


root = 'H3N2 NA'
n_reps = 2

for f in [os.path.join(FREQ_DIR, '_'.join([root, 'single_aa_rep_%d.csv' % i])) for i in range(1, 1+n_reps)]:
    df = pd.read_csv(f)
    for df_iter, df_entry in df.iterrows():
        if df_entry['aa']=='-':
            df.loc[df_iter, 'aa'] = '*'
    df.to_csv(f + '.gz', index=False, compression='gzip')

for f in [os.path.join(SEL_DIR, '_'.join([root, 'selection_coefficients.csv']))]:
    df = pd.read_csv(f)
    df.to_csv(f + '.gz', index=False, compression='gzip')

for f in [os.path.join(OUT_DIR + 'merged_preference/', root+'.csv')]:
    df = pd.read_csv(f)
    df.to_csv(f + '.gz', index=False, compression='gzip')

In [2]:
help(popDMS.compute_variant_frequencies)

Help on function compute_variant_frequencies in module popDMS:

compute_variant_frequencies(name, reference_sequence_file, haplotype_counts_file, n_replicates, time_points, time_cols, freq_dir='.', with_epistasis=False, comment_char=None, format='MaveDB')
    Computes variant (allele) frequencies, used by popDMS, from haplotype counts. By
    default, we assume that counts are saved in MaveDB format. For more information
    on this format, see https://www.mavedb.org/docs/mavedb/data_formats.html
    
    Required arguments:
        - name: String that will be associated with saved files, serving as an
            identifier for the data set
        - reference_sequence_file: File path to the reference sequence, which is
            needed to specify 'WT' variants
        - haplotype_counts_file: Path to an existing file recording haplotype
            frequencies
        - n_replicates: Number of experimental replicates
        - time_points: Time points in the data; for normalization

In [3]:
help(popDMS.infer_correlated)

Help on function infer_correlated in module popDMS:

infer_correlated(name, n_replicates, corr_cutoff_pct, freq_dir='.', output_dir='.', with_epistasis=False, plot_gamma=True)
    Infer selection coefficients from data that includes correlations (counts)
    between pairs of mutant sites. This is not possible with data that includes
    single codon/amino acid counts only.
    
    Required arguments:
        - name: String that will be associated with saved files, serving as an
            identifier for the data set
        - n_replicates: Number of experimental replicates
        - corr_cutoff_pct: Cutoff on the maximum allowed drop in correlation
            between replicates, used to determine the optimal regularization
            strength
    
    Optional arguments:
        - freq_dir (default: '.'): Directory where variant frequency data has been 
            saved; assumes data saved in the format of compute_variant_frequencies
        - output_dir (default: '.'): Directory 

In [4]:
help(popDMS.compute_variant_frequencies_Bloom)

Help on function compute_variant_frequencies_Bloom in module popDMS:

compute_variant_frequencies_Bloom(name, codon_counts_files, replicates, times, freq_dir='.', error_file=None, comment_char=None)
    Compute variant frequencies from codon counts in Bloom lab format. If an error
    file is provided, corrects for sequencing errors.
    
    Required arguments:
        - name: Name of the experiment
        - codon_counts_files: List of paths to codon counts files
        - replicates: List of replicate numbers
        - times: List of time points
        - freq_dir: Path to directory where frequencies will be saved
    
    Optional arguments:
        - error_file (default: None): Path to a file recording sequence counts from
            sequencing the reference sequence only, used to estimate sequencing
            error rates and correct variant frequencies
        - comment_char (default: None): Character used to indicate comments in the
            codon counts files



In [5]:
help(popDMS.infer_independent)

Help on function infer_independent in module popDMS:

infer_independent(name, n_replicates, corr_cutoff_pct, freq_dir='.', output_dir='.', plot_gamma=True)
    Infer selection coefficients from data that consists of single amino acid
    frequencies only.
    
    Required arguments:
        - name: String that will be associated with saved files, serving as an
            identifier for the data set
        - n_replicates: Number of experimental replicates
        - corr_cutoff_pct: Cutoff on the maximum allowed drop in correlation
            between replicates, used to determine the optimal regularization
            strength
    
    Optional arguments:
        - freq_dir (default: '.'): Directory where variant frequency data has been
            saved; assumes data saved in the format of compute_variant_frequencies
        - output_dir (default: '.'): Directory where selection coefficients will be
            saved
        - plot_gamma (default: True): Plot correlation between rep

In [9]:
%%time

reload(popDMS)

barcode_data = dict(
    name = 'test',
    replicate_files = ['sample_barcode_counts_small.csv'],
    output_dir = '.',
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_barcode(**barcode_data)

4991
Only one replicate, setting gamma=1
CPU times: user 18.6 s, sys: 640 ms, total: 19.2 s
Wall time: 2.48 s


# YAP1

In [ ]:
%%time

reload(popDMS)

name = 'YAP1'
n_replicates = 2

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'YAP1_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'YAP1_aa2codon_counts.csv.gz'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 1, 2, 3],
    time_cols = {1: ['101208_c_0', '101208_c_1', '101208_c_2', '101208_c_3'],  
                 2: ['110307_c_0', '110307_c_1', '110307_c_2', '110307_c_3']},
    comment_char = None
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# TpoR MPL

In [ ]:
%%time

name = 'TpoR'
n_replicates = 6

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'TpoR_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'TpoR_nucleotide_counts.csv'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 1],
    time_cols = {1: ['Replicate_A_c_0', 'Replicate_A_c_1'],
                 2: ['Replicate_B_c_0', 'Replicate_B_c_1'],
                 3: ['Replicate_C_c_0', 'Replicate_C_c_1'],
                 4: ['Replicate_D_c_0', 'Replicate_D_c_1'],
                 5: ['Replicate_E_c_0', 'Replicate_E_c_1'],
                 6: ['Replicate_F_c_0', 'Replicate_F_c_1']},
    comment_char = '#'
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# TpoR MPL S505N

In [ ]:
%%time

name = 'TpoR_S505N'
n_replicates = 6

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'TpoR_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'TpoR_S505N_nucleotide_counts.csv'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 1],
    time_cols = {1: ['Replicate_1_c_0', 'Replicate_1_c_1'],
                 2: ['Replicate_2_c_0', 'Replicate_2_c_1'],
                 3: ['Replicate_3_c_0', 'Replicate_3_c_1'],
                 4: ['Replicate_4_c_0', 'Replicate_4_c_1'],
                 5: ['Replicate_5_c_0', 'Replicate_5_c_1'],
                 6: ['Replicate_6_c_0', 'Replicate_6_c_1']},
    comment_char = '#'
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# Ube4b

In [ ]:
%%time

name = 'Ube4b'
n_replicates = 2

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'Ube4b_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'Ube4b_nucleotide_counts.csv'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 1, 2, 3],
    time_cols = {1: ['Rep_2_c_0', 'Rep_2_c_1', 'Rep_2_c_2', 'Rep_2_c_3'],
                 2: ['Rep_3_c_0', 'Rep_3_c_1', 'Rep_3_c_2', 'Rep_3_c_3']},
    comment_char = '#'
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# BRCA1

In [ ]:
%%time

reload(popDMS)

name = 'Y2H_1'
n_replicates = 3

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'BRCA1_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'BRCA1_nucleotide_counts.csv'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 18, 37, 45],
    time_cols = {1: ['Y2H_1_Rep1_c_0', 'Y2H_1_Rep1_c_18', 'Y2H_1_Rep1_c_37', 'Y2H_1_Rep1_c_45'],
                 2: ['Y2H_1_Rep2_c_0', 'Y2H_1_Rep2_c_18', 'Y2H_1_Rep2_c_37', 'Y2H_1_Rep2_c_45'],
                 3: ['Y2H_1_Rep3_c_0', 'Y2H_1_Rep3_c_18', 'Y2H_1_Rep3_c_37', 'Y2H_1_Rep3_c_45']},
    comment_char = '#'
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

In [ ]:
%%time

reload(popDMS)

name = 'Y2H_2'
n_replicates = 3

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'BRCA1_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'BRCA1_nucleotide_counts.csv'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 16, 41, 64],
    time_cols = {1: ['Y2H_2_Rep1_c_0', 'Y2H_2_Rep1_c_16', 'Y2H_2_Rep1_c_41', 'Y2H_2_Rep1_c_64'],
                 2: ['Y2H_2_Rep2_c_0', 'Y2H_2_Rep2_c_16', 'Y2H_2_Rep2_c_41', 'Y2H_2_Rep2_c_64'],
                 3: ['Y2H_2_Rep3_c_0', 'Y2H_2_Rep3_c_16', 'Y2H_2_Rep3_c_41', 'Y2H_2_Rep3_c_64']},
    comment_char = '#'
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

In [ ]:
%%time

reload(popDMS)

name = 'E3'
n_replicates = 6

count_data = dict(
    name = name,
    reference_sequence_file = os.path.join(RAW_DIR, 'BRCA1_reference_sequence.dat'),
    haplotype_counts_file = os.path.join(RAW_DIR, 'BRCA1_nucleotide_counts.csv'), 
    freq_dir = FREQ_DIR,
    n_replicates = n_replicates,
    time_points = [0, 1, 2, 3, 4, 5],
    time_cols = {1: ['PlusE2NewRep3_c_0', 'PlusE2NewRep3_c_1', 'PlusE2NewRep3_c_2', 'PlusE2NewRep3_c_3', 'PlusE2NewRep3_c_4', 
                     'PlusE2NewRep3_c_5'],
                 2: ['PlusE2NewRep4_c_0', 'PlusE2NewRep4_c_1', 'PlusE2NewRep4_c_2', 'PlusE2NewRep4_c_3', 'PlusE2NewRep4_c_4', 
                     'PlusE2NewRep4_c_5'],
                 3: ['PlusE2NewRep5_c_0', 'PlusE2NewRep5_c_1', 'PlusE2NewRep5_c_2', 'PlusE2NewRep5_c_3', 'PlusE2NewRep5_c_4', 
                     'PlusE2NewRep5_c_5'],
                 4: ['PlusE2Rep3_c_0', 'PlusE2Rep3_c_1', 'PlusE2Rep3_c_2', 'PlusE2Rep3_c_3', 'PlusE2Rep3_c_4', 'PlusE2Rep3_c_5'],
                 5: ['PlusE2Rep4_c_0', 'PlusE2Rep4_c_1', 'PlusE2Rep4_c_2', 'PlusE2Rep4_c_3', 'PlusE2Rep4_c_4', 'PlusE2Rep4_c_5'],
                 6: ['PlusE2Rep5_c_0', 'PlusE2Rep5_c_1', 'PlusE2Rep5_c_2', 'PlusE2Rep5_c_3', 'PlusE2Rep5_c_4', 'PlusE2Rep5_c_5']},
    comment_char = '#'
)
popDMS.compute_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# DBR1

In [ ]:
%%time

reload(popDMS)

name = 'DBR1'
n_replicates = 2

count_data = dict(
    dir = FREQ_DIR,
    name = name,
    types = ['single', 'double'],
    n_replicates = n_replicates,
    reference_sequence_file = os.path.join(RAW_DIR, name + '_reference_sequence.dat'),
    comment_char = None
)
popDMS.convert_codon_counts_to_variant_frequencies(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    with_epistasis = False,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# SARS-CoV-2 Spike

In [ ]:
%%time

reload(popDMS)

name = 'Spike'
n_replicates = 2

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_correlated(**selection_data)

# Independent (no mutant-mutant correlations) proteins

# Zika virus

In [ ]:
%%time

reload(popDMS)

name = 'ZIKV'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza Perth 2009 (H3N2)

In [ ]:
%%time

reload(popDMS)

name = 'Perth2009'
n_replicates = 4

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza WSN

In [ ]:
%%time

reload(popDMS)

name = 'WSN'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza PR8

In [ ]:
%%time

reload(popDMS)

name = 'PR8'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza Aichi 68C

In [ ]:
%%time

reload(popDMS)

name = 'Aichi68C'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza Matrix_M1

In [ ]:
%%time

reload(popDMS)

name = 'Matrix_M1'
filename = 'HomM1'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza A549

In [ ]:
%%time

reload(popDMS)

name = 'A549'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza CCL141

In [ ]:
%%time

reload(popDMS)

name = 'CCL141'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza MS

In [ ]:
%%time

reload(popDMS)

name = 'MS'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza MxA

In [ ]:
%%time

reload(popDMS)

name = 'MxA'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza MxAneg

In [ ]:
%%time

reload(popDMS)

name = 'MxAneg'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (name, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % name),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Influenza H3N2 NA

In [ ]:
%%time

reload(popDMS)

name = 'H3N2 NA'
n_replicates = 2

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV Env BG505

In [ ]:
%%time

reload(popDMS)

name = 'HIV Env BG505'
filename = 'BG505'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV Env BF520

In [ ]:
%%time

reload(popDMS)

name = 'HIV Env BF520'
filename = 'BF520'
n_replicates = 3

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV Env BF520 human host

In [ ]:
%%time

reload(popDMS)

name = 'HIV BF520 human'
filename = 'human'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV Env BF520 rhesus host

In [ ]:
%%time

reload(popDMS)

name = 'HIV BF520 rhesus'
filename = 'rhesus'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV bnAbs FP16

In [ ]:
%%time

reload(popDMS)

name = 'HIV bnAbs FP16'
filename = 'FP16'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV bnAbs FP20

In [ ]:
%%time

reload(popDMS)

name = 'HIV bnAbs FP20'
filename = 'FP20'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# HIV bnAbs VRC34

In [ ]:
%%time

reload(popDMS)

name = 'HIV bnAbs VRC34'
filename = 'VRC34'
n_replicates = 2

count_data = dict(
    name = name,
    codon_counts_files = [os.path.join(RAW_DIR, '%s_mutDNA-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)] + \
                         [os.path.join(RAW_DIR, '%s_mutvirus-%d_codoncounts.csv' % (filename, i+1)) for i in range(n_replicates)],
    replicates = [i+1 for i in range(n_replicates)] + [i+1 for i in range(n_replicates)],
    times = [0 for i in range(n_replicates)] + [1 for i in range(n_replicates)],
    freq_dir = FREQ_DIR,
    # error_file = os.path.join(RAW_DIR, '%s_DNA_codoncounts.csv' % filename),
    comment_char = None
)
popDMS.compute_variant_frequencies_Bloom(**count_data)

selection_data = dict(
    name = name,
    freq_dir = FREQ_DIR,
    output_dir = SEL_DIR,
    n_replicates = n_replicates,
    corr_cutoff_pct = CORR_CUTOFF_PCT,
    plot_gamma = True
)
popDMS.infer_independent(**selection_data)

# Comparing enrichment ratios and selection coefficients at example sites

In [ ]:
import numpy as np
import pandas as pd

name = 'HIV Env BG505'
filename = 'BG505'
n_replicates = 3
gamma = 2e-3

ex_sites = [        592,        596,         508,         525]
ex_aas   = [['I', 'L',], ['S', 'F'], ['G', 'R', 'Y', 'H', 'F', 'K'], ['S', 'N',]]

df = [pd.read_csv(os.path.join(FREQ_DIR, name + ('_single_aa_rep_%d.csv.gz' % (rep+1)))) for rep in range(n_replicates)]

for i_site in range(len(ex_sites)):
    print('%s site %d' % (name, ex_sites[i_site]))
    print('residue\treplicate\tx0\t\tx1\t\tratio\t\td log\t\td logit\t\traw s\t\treg s')

    for a in ex_aas[i_site]:
        for i_rep in range(n_replicates):
            df_temp = df[i_rep][(df[i_rep]['site']==ex_sites[i_site]) & (df[i_rep]['aa']==a)]
            x0 = df_temp[df_temp['generation']==0].iloc[0]['frequency']
            x1 = df_temp[df_temp['generation']==1].iloc[0]['frequency']
            
            xvar   = ((x0+x1)/2) * (1 - ((x0+x1)/2)) 
            ratio  = x1/x0
            dlog   = np.log(x1/x0)
            dlogit = np.log(x1/(1-x1))-np.log(x0/(1-x0))
            s_raw  = (x1 - x0) / xvar
            s_reg  = (x1 - x0) / (xvar + gamma)

            res_str = a if i_rep==0 else ''
            print('%s\t%d\t\t%.3e\t%.3e\t%.3e\t%.3e\t%.3e\t%.3e\t%.3e' % (res_str, i_rep+1, x0, x1, ratio, dlog, dlogit, s_raw, s_reg))

    print('')

In [ ]:
import numpy as np
import pandas as pd

name = 'Ube4b'
n_replicates = 2
gamma = 2.8e-04

ex_sites = [             34]
ex_aas   = [['A', 'D', 'N']]

df = [pd.read_csv(os.path.join(FREQ_DIR, name + ('_single_aa_rep_%d.csv.gz' % (rep+1)))) for rep in range(n_replicates)]

for i_site in range(len(ex_sites)):
    print('%s site %d' % (name, ex_sites[i_site]))
    print('residue\treplicate\tx0\t\tx1\t\tratio\t\td log\t\td logit\t\traw s\t\treg s')

    for a in ex_aas[i_site]:
        for i_rep in range(n_replicates):
            df_temp = df[i_rep][(df[i_rep]['site']==ex_sites[i_site]) & (df[i_rep]['aa']==a)]
            x = [df_temp[df_temp['generation']==i].iloc[0]['frequency'] for i in range(4)]

            xvar   = [((x[i]+x[i+1])/2) * (1 - ((x[i]+x[i+1])/2)) for i in range(len(x)-1)] 
            ratio  = x[-1]/x[0]
            dlog   = np.log(x[-1]/x[0])
            dlogit = np.log(x[-1]/(1-x[-1]))-np.log(x[0]/(1-x[0]))
            s_raw  = (x[-1] - x[0]) / np.sum(xvar)
            s_reg  = (x[-1] - x[0]) / (np.sum(xvar) + gamma)

            res_str = a if i_rep==0 else ''
            print('%s\t%d\t\t%.3e\t%.3e\t%.3e\t%.3e\t%.3e\t%.3e\t%.3e' % (res_str, i_rep+1, x[0], x[-1], ratio, dlog, dlogit, s_raw, s_reg))

    print('')